In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey
import json
import os
import re
from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import sim_loader as sim_load
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:
# Identify which simulation
folder_path = "sims/gas_acc/test_folder"
timestep = 100
H_r_model = 'Lambrechts_mixed'
e_el = 0.01

# List all files in the given folder
all_files = os.listdir(folder_path)
# create the list of names of the sim, sim_params and params files 
sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model+'_e_el_'+str(e_el)) and f.endswith('N_steps'+str(timestep)+'.json')]
sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model+'_e_el_'+str(e_el)) and f.endswith('N_steps'+str(timestep)+'.json')]
params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model+'_e_el_'+str(e_el)) and f.endswith('N_steps'+str(timestep)+'.json')]

# Sort the filenames based on the initial time
sim_filenames.sort(key=sim_load.extract_initial_time)
sim_params_filenames.sort(key=sim_load.extract_initial_time)
params_filenames.sort(key=sim_load.extract_initial_time)


# Load the simulations, sim_params, and params
simulations = [sim_load.JSONtoSimRes(filename) for filename in sim_filenames]
sim_parameters = [sim_load.load_sim_params(filename) for filename in sim_params_filenames]
parameters = [sim_load.load_params(filename) for filename in params_filenames]



fig, axs = plt.subplots(1,3, figsize=(18, 6))
title = ["Irradiation", "Surface heating", "Midplane heating"]
a_p0 = np.geomspace(5e-3, 1e2, num = 1000)

alpha_transp = 0.2

############### DISCS loops ########################
for k, (simulations, parameters, sim_parameters) in enumerate(zip([simulations], [parameters], [sim_parameters])):
    for j in range(len(simulations)):
        sim = simulations[j]
        params = parameters[j]
        sim_params = sim_parameters[j]
        for p in range(1,sim_params.nr_planets):
            print(sim_params.nr_planets)
            print("sim mass size", sim.mass[p].size)
            idx = plot.idxs (axs[k], sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            cmap =  mpl.cm.inferno.reversed()
            norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
            colors = cmap(norm(sim_params.t0[p]))
            print("start time embryos", sim_params.t0)
            axs[k].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
            cmap = cmap)

            if iso_idx != 0:
                axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
                                color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

            axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                        color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
            
        for i in range(1):
            for p in [0]:
                idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                                sim.dR_dt[p], sim.dM_dt[p], params, True)
                iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
                stop_mig_idx = idx['stop_mig_idx'].values[0]
                dt = (sim.time[:iso_idx]).to(u.Myr)
                sc = axs[k].scatter(sim.position[p,:iso_idx].to(u.au), sim.mass[p,:iso_idx].to(u.M_earth), c = dt, 
                                    norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), 
                cmap = mpl.cm.inferno.reversed())
                axs[k].loglog(sim.position.to(u.au)[p,iso_idx:stop_mig_idx], sim.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, 
                            alpha = alpha_transp)
                axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
                axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.earthMass), 
                            color = cmap(norm(sim.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
                if j == 0:
                    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
                    cbar = fig.colorbar(sc, cax=cbar_ax)	
                    # Manually set the colorbar boundaries and ticks
                    cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0,2.0, 5.0]))
                    cbar_ax.yaxis.set_major_formatter(LogFormatter())
                    cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
                    cbar.set_label('accretion time [Myr]', fontsize=20, labelpad=15)
                    cbar.ax.tick_params(axis = 'both', which = 'major', size = 18, labelsize = 18)
                    cbar.ax.tick_params(axis = 'both', which = 'minor', size = 12)

        axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[k].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)

    print(k, planet_counter(simulations, parameters, sim_parameters))

#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker

for j in range(3):
    # #SE
    axs[j].fill_betweenx([1, 20], 0.001, 1, color='brown', alpha=0.1)
    #terrestrial embryo
    axs[j].fill_betweenx([0.01, 1], 0.1, 10, color='black', alpha=0.1)

    # gas giants
    axs[j].fill_betweenx([1e2, 6e3], 0.001, 0.1, color='aquamarine', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 0.1, 2, color='gold', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 2, 10, color='purple', alpha=0.1)


    plot.HD219134(axs[j], color = 'red')
    plot.solar_system(axs[j], color = 'green')
    params = parameters[0]
    sim_params = sim_parameters[0]

    axs[j].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

    axs[j].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[j].set_title(title[j], fontsize = 20)
    axs[j].set_xlim(5e-3, 1e2)
    axs[j].tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
    axs[j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[j].set_ylim(5e-8, 1e3)

    axs[j].yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs[j].yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))
    plot.all_x_ticks(axs[j], num_ticks=100)

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_gas_surfheat_5Myr", bbox_inches='tight')